# Halfmoon

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.0


In [3]:
import pandas as pd
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier


import os
#os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [5]:
n = 600

df = pd.read_csv(f'../data/halfmoon/halfmoon_{n}_train.csv')
x_train = df.drop('Target', axis=1)
y_train = df['Target']
print(len(x_train))

300


In [6]:
x_train

,Feature_1,Feature_2
0,0.067785,0.210961
1,-0.034452,0.322632
2,1.649217,-0.319410
3,1.927635,0.030436
4,0.188512,0.145691
...,...,...
295,0.810149,0.415593
296,0.732141,0.664749
297,-0.030117,0.771831
298,-0.845699,0.349156


In [7]:
llm_model_name = 'gpt-4'

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model_name=llm_model_name, params=params)

### Training

In [11]:
file_path = '../models/halfmoon/'

print(f'Number of data:{len(x_train)}')
model = iblm.fit(x_train, y_train, model_name = 'halfmoon', file_path=file_path)

Number of data:300
> Start of model creating.
Tokens Used: 7056
	Prompt Tokens: 6892
	Completion Tokens: 164
Successful Requests: 1
Total Cost (USD): $0.2166


In [12]:
# Code of the model created
print(model)

import numpy as np

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.

        # Calculate the distance from the origin (0, 0)
        distance = np.sqrt(row['Feature_1']**2 + row['Feature_2']**2)

        # Normalize the distance to a range between 0 and 1
        normalized_distance = distance / (np.sqrt(2) * 2)

        # Calculate the probability of the target being 1
        y = 1 - normalized_distance

        # Do not change the code after this point.
        output.append(y)
    return np.array(output)


### Prediction

In [13]:
df = pd.read_csv(f'../data/halfmoon/halfmoon_{n}_test.csv')
x_test = df.drop('Target', axis=1)
y_test = df['Target']

In [14]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [15]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.30666666666666664
Precision: 0.38016528925619836
Recall: 0.6133333333333333
F1 score: 0.4693877551020408
ROC-AUC: 0.44266666666666665
